In [1]:
from itertools import chain

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import scipy


from scipy.sparse.csgraph import laplacian

from sklearn.cluster import SpectralClustering
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import rbf_kernel, cosine_similarity
from sklearn.cluster import SpectralClustering
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix, make_scorer, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics.pairwise import rbf_kernel, cosine_similarity, kernel_metrics
from sklearn.metrics import pairwise_distances

from sklearn.model_selection import train_test_split

import feature, utils

In [2]:
# Load data
df_data = pd.read_csv('../data/ice-cat-office-products.csv.gz', dtype=str, index_col=0)

category_counts = df_data["category_name"].value_counts()
large_enough_categories = category_counts[category_counts > 20].index.tolist()

df_data = df_data[df_data.category_name.isin(large_enough_categories)]

print(f"Number of categories: {len(df_data.category_name.unique())}")

Number of categories: 69


In [3]:
feateure_transformer = feature.BasicIceCatFeatureTransformer(output_size=128)

In [4]:
X = feateure_transformer.fit_transform(df_data)

In [5]:
D = pairwise_distances(X, metric="euclidean")
D_normalized = D / np.max(D)
K = 1 - D_normalized

In [6]:
n_labels = len(df_data.category_name.unique())

clustering = SpectralClustering(
    n_clusters=2*n_labels,
    assign_labels='discretize',
    affinity='precomputed',
    random_state=42,
    verbose=10
)

In [7]:
clustering.fit(K)

Computing label assignment using discretize


SpectralClustering(affinity='precomputed', assign_labels='discretize',
                   n_clusters=138, random_state=42, verbose=10)

In [8]:
df_data['cluster'] = clustering.labels_
df_data['cluster'] = df_data['cluster'].astype(str)

In [9]:
df_data = utils.combined_similar_clusters(df_data)

In [10]:
print(classification_report(y_true=df_data.category_name, y_pred=df_data.pred_category))

                                    precision    recall  f1-score   support

                    Ballpoint Pens       0.00      0.00      0.00        21
                    Binding Covers       0.37      0.33      0.35       190
                 Board Accessories       0.34      0.68      0.45       154
                   Bulletin Boards       0.00      0.00      0.00       123
                       Calculators       0.99      0.91      0.95       305
       Charging Station Organizers       1.00      0.39      0.56        31
                    Colour Pencils       0.51      0.47      0.49        38
                    Computer Desks       0.19      0.69      0.30       144
          Computer Furniture Parts       0.00      0.00      0.00        33
                           Crayons       0.00      0.00      0.00        27
            Desk Drawer Organizers       0.58      0.57      0.57        44
             Desk Trays/Organizers       1.00      0.09      0.17       159
           

/home/omar/.cache/pypoetry/virtualenvs/snow-leopard-O1gqMmLm-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/omar/.cache/pypoetry/virtualenvs/snow-leopard-O1gqMmLm-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/omar/.cache/pypoetry/virtualenvs/snow-leopard-O1gqMmLm-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_

In [11]:
cols = ["cluster", "pred_category", "category_name"] 
df_data[df_data.pred_category != df_data.category_name][cols]

,cluster,pred_category,category_name
2,65,Computer Desks,Fax Machines
4,65,Computer Desks,Staples
5,120,Printer Cabinets & Stands,Paper Cutters
6,120,Printer Cabinets & Stands,Paper Cutters
7,65,Computer Desks,Printer Cabinets & Stands
...,...,...,...
7961,4,Board Accessories,Bulletin Boards
7962,4,Board Accessories,Bulletin Boards
7963,4,Board Accessories,Bulletin Boards
7964,4,Board Accessories,Bulletin Boards
